## 7.3 고수준 API

layer 모듈을 이용하여 앞서 만든 CNN 모델을 조금 더 간단하게 만듦!

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)

C:\Users\soual\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [10]:
X = tf.placeholder(tf.float32, [None, 28,28,1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool) #dropout 사용하는 경우

In [11]:
L1 = tf.layers.conv2d(X, 32, [3,3], activation = tf.nn.relu, padding = 'SAME')
L1 = tf.layers.max_pooling2d(L1, [2,2], [2,2], padding = "SAME")
L1 = tf.layers.dropout(L1, 0.7, is_training)

첫 번째 convolution 및 pooling layer

원래는

<pre>
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01))
L1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = 'SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
<code>

위와 같이 tf.layers module을 사용하면 줄일 수 있음!

In [12]:
L2 = tf.layers.conv2d(L1, 64, [3,3], activation = tf.nn.relu, padding = 'SAME')
L2 = tf.layers.max_pooling2d(L2, [2,2], [2,2], padding = 'SAME')
L2 = tf.layers.dropout(L2, 0.7, is_training)

In [13]:
L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation = tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

tf.layers module을 사용하면 간단하게 작성 가능함

-> 직접 생각하고 계산해야 하는 것들 생략 가능

tf.layers와 같은 고급 API 사용

-> activation 함수나 convolution layer 만들기 위한 나머지 수치를 알아서 계산하고 적용

-> weight 초기화시 기본적으로 xavier_initializer 함수 사용 

=> 신경망을 효율적으로 만들어주는 옵션들 사용!

< 앞에서 만든 코드와 비교해보세요~>

< 뉴런과 계층수를 늘리거나 hyperparameter 변경하며 정확도를 높여보세요~>

In [14]:
model = tf.layers.dense(L3, 10, activation=None)

In [15]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [ ]:
# 신경망 모델 학습
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i  in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1,28,28,1) #model의 input으로 넣기 위해 수정
        _, cost_val = sess.run([optimizer, cost], feed_dict = {X: batch_xs, Y: batch_ys, is_training: True})
        
        
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch+1), 'Avg. cost = ', '{:.4f}'.format(total_cost/total_batch))
    
print('최적화 완료!')

#결과 확인
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: ', sess.run(accuracy, feed_dict = {X: mnist.test.images.reshape(-1,28,28,1), Y: mnist.test.labels, is_training: False}))

Epoch: 0001 Avg. cost =  0.1626
Epoch: 0002 Avg. cost =  0.0454
Epoch: 0003 Avg. cost =  0.0301
Epoch: 0004 Avg. cost =  0.0218
